In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
#generowanie sztucznych danych czasowych
def generate_data(n_samples, n_features, n_timesteps):
    X = np.random.rand(n_samples, n_features, n_timesteps)
    y = np.sum(X, axis=1) + np.random.rand(n_samples, n_timesteps) * 0.1
    return X, y

In [12]:
n_samples = 1000
n_features = 10
n_timesteps = 10
X, y = generate_data(n_samples, n_features, n_timesteps)

In [13]:
#normalizacja danych
scaler = MinMaxScaler()
X = X.reshape(-1,n_features)
X = scaler.fit_transform(X)
X = X.reshape(n_samples,n_timesteps,n_features)

y = scaler.fit_transform(y)


In [14]:
#podział danych
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#budowa modelu regresyjnego
model = Sequential()
model.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_features))

model.compile(loss='mse', optimizer=Adam())

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)                    │ (None, 9, 64)               │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 50)                  │          12,850 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             510 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,704 (57.44 KB)

 Trainable params: 14,704 (57.44 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1748 - val_loss: 0.0379
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0334 - val_loss: 0.0293
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0278 - val_loss: 0.0268
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0247 - val_loss: 0.0249
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0224 - val_loss: 0.0233
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0207 - val_loss: 0.0217
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0191 - val_loss: 0.0197
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0171 - val_loss: 0.0182
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0156 - val_loss: 0.0169
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0145 - val_loss: 0.0156
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0131 - val_loss: 0.0146
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0120 - val_l

In [17]:
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0062 
Test loss: 0.006263048853725195


In [19]:
predictions = model.predict(X_test)
print(predictions[:5])
print(y_test[:5])

print(np.argmax(predictions[:5]))
print(np.argmax(y_test[:5]))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[0.36357054 0.46706402 0.4576242  0.7887635  0.55602825 0.5088994
  0.55564183 0.560273   0.6967068  0.79370993]
 [0.4986998  0.3426414  0.67902386 0.77622914 0.5391774  0.4702587
  0.6554611  0.7415334  0.6007919  0.2761002 ]
 [0.29529536 0.45224494 0.4726071  0.38183212 0.28242624 0.6199629
  0.46355546 0.47309262 0.5461981  0.63671494]
 [0.49425954 0.6554934  0.19597882 0.44588023 0.5583111  0.5829354
  0.5643401  0.61840266 0.2817498  0.79813874]
 [0.54849386 0.5393389  0.62746805 0.3065198  0.34389654 0.6755164
  0.87308013 0.5869084  0.41406485 0.32489908]]
[[0.46016547 0.55807576 0.42945392 0.63049362 0.61011857 0.47952778
  0.54467996 0.55573525 0.7758288  0.73994656]
 [0.52764585 0.385234   0.522559   0.73216151 0.5276592  0.44177518
  0.68981947 0.76682955 0.74937809 0.20417123]
 [0.18404341 0.34739054 0.3952859  0.37418649 0.3126437  0.51783028
  0.51765421 0.45718949 0.57048507 0.64634357]
 [0.48619618 0.64962466 0.02471658 0.38216687 